# DSCI 6003 5.2 Lecture

## By the end of this lecture you will be able to:
1. Be familiar with the concept of boosting
2. Be able to describe the algorithm of Adaptive Boosting in your own words
3. Be able to describe the algorith of Gradient Boosting in your own words

## Boosting in General

You will often hear the word "Bagging" dropped around the same time as "Boosting." The two are very different techniques and not really used in the same context. You already know what bagging is. 

Boosting, properly stated, the use of the output of one model as the input of another. Thus we can use Pipelines to construct whole groups of models that are technically "boosted" by each other. Any and all models can be boosted, according to the theory. 

This aspect of model construction is commonly employed by larger data science work groups (sometimes called "enterprise scale solutions" instead of "boosted models"), but don't get bugged out by the jargon.

The word "boosting" is most commonly used to describe boosting used with trees, and most particularly Gradient Boosted Regression Trees (GBRTs). The patois used to describe it is something along the lines of "combining weak learners" to make a "strong learner". This is mumbo-jumbo. The "weak" learners are just normal decision trees (as opposed to an estimator which is especially "weak" in some way). 

**Hypothesis:** Same as base "weak" learner.

**Cost:** to include the cost of the base learner, as well as a cost function reflecting the residual error from successive fits.

**Optimization:** Either adaptive optimization (reinforcement model) or gradient descent.

## Boosting in Trees

There are two types of boosting used in trees in common practice today. This is the most common jargon usage of this word in data science. If you hear fellow data scientists say "boosting", they are often referring to boosted trees.

### Adding bias into the training process

On your last skills test we discussed employing a very simplified version of Extremely Random Trees. These are only sometimes effective and only when highly pruned and in a forest, however they do capture an interesting principle, and that is that each branch of the trees is sensitive to the specialized relationships among the k random variables. In this sense, they select against noise and reduce variance at the cost of some bias.

This principle is used directly in the construction of boosted trees. In the boosted tree algorithms, a "family" of trees is "grown" over successive generations of trees, by making each successor tree an expert on attacking the weaknesses of the other. What results is a single "master" tree designed to defeat all weaknesses of the training set selection. Members of the family are weighted by the ratio of errors not covered by the previous members. 

The process of boosting (in any case) is based on measuring the degree to which a given tree fails in making a good prediction. This, of course, is the cost or more commonly in the case of trees "loss" function.

### Some notes about the boosting community
1. They try to make algorithms sound like products ("LogitBoost","AnyBoost", etc.)
2. "loss function" = cost function
3. "weak learners" = normal estimators
4. "model is complex/review of the theory" = we don't understand it very well/it's all Black Magic to us

# AdaBoost

AdaBoost is the simpler of the two boosting learners we shall discuss. It focuses on providing each tree a description of the mistakes of the last tree (through the cost "loss" function). Each tree is given the opportunity to **Ada**pt itself to the weaknesses of previous trees. Finally the end classifier is something like a random forest, where each of the trees votes, weighted inversely to its own total error.

![adaboost_scheme](./images/AdaBoost_scheme.png)

Below is shown a plot of the AdaBoost partition:

![adaboost_part](./images/AdaBoost_part.png)


Successive trees end up covering the limitations of previous trees. The size of the points reflect their weighting at that point. The misclassified points are *individually* penalized, just as misclassified points are penalized in SVMs. 

The next tree in the family may or may not correctly classify these points (it is still just a simple decision tree), but if it does, then the total cost function comes down, and it is more favorable to keep the tree. If it does not, the weights continue to increase until a tree comes along that correctly classifies these points. The weights $w_{i}$ are used in the impurity estimator during the split, making it progressively more unfavorable when we calculate the information gain if they are misclassified (fall inside a leaf belonging to another class - the weights increase the equivalent counts belonging to that class).

Thus the most accurate trees (at the end of the boost chain) are often the most dilute (reflect the fewest points), but they are weighted (by error score) much more than the earliest. The mixture of different trees can manage outliers and noise much better than otherwise.


## The AdaBoost Algorithm

    Fit:
    
    Initialize weights W to be 1/N
    for m in range(1,M): (M determined by user)
        fit a standard decision tree G(X,w)
        compute total error err <- sum(W*misclassified points)/sum(W)
        compute alpha_i <- log((1-err)/err)
        change weights by an exponent if misclassified w_i <- 
        w_i * exp(alpha_i)
   
    Predict:
     
    
    for m in range(1, M):
        sum += alpha_m * G_m(X)
    return sign(sum)

## QUIZ:

Can you think of another algorithm that might benefit from adaptive boosting?

# Gradient Boosting

Gradient boosting (GB) can be done with many classifiers, although in this case we are discussing its use with decision trees. It is significantly different from AdaBoost in that it attempts to create a single tree rather than a forest. The gradient boost tree is evolved over successive generations by enabling each new generation of tree to **predict the gradient of the cost function - meaning the error!** of the previous generation. 

The cost function used involves calculating the residuals; the gradient is another function taken from the cost function (quite like that of logistic regression). 

This matter of the cost function and why this idea works is relatively complex one. A [long history](http://www.ncbi.nlm.nih.gov/pmc/articles/PMC3885826/) exists surrounding it. Historically, GB evolved from relatively arbitrary experiments with methods of this kind in the late 90's, and much of the material (and code) surrounding it exists in an essentially experimental state. Later years saw some theoretical explanation develop surrounding it, but most of this is still rather nebulous in nature.

FYI: The cost function most commonly used today differs by the type of data being classified/regressed on. For continuous data it is the Huber loss function. Bernoulli loss is normally used for categorical variables.  

![gradient_boosting_scheme](./images/GradientBoosting_scheme.png)


## Gradient Boosting Algorithm for Trees

    Fit:

    Initialize first decision tree and set residuals (r) <- labels
        for m in xrange(1,M): #set by user
            fit a new tree on (X, r) 
            calculate a lagrange multiplier using line search (argmin) on the loss function
            the new tree is an update to the old tree's leaves by computing a gradient descent step (Newton-Raphson) in the direction of the gradient, using the lagrange multiplier as a weight. This changes the values/classes of the leaf in the direction of the gradient (cost function).
            recompute the residuals with the newly calculated tree.

     produces a final boosted model.


    Predict:
        
        as a normal tree

## QUIZ:
Does it seem natural to you that we can fit on the error of a model?

## Why does any of this work?

Most of the machine learning community seems baffled by the success of this approach, and in general, GB tends to outperform even very sophisticated methods, including neural networks. 

Why? My intuition is that the algorithm ends up partitioning on the noise rather than the signal. What I think happens is that signal gets partitioned into a few relatively powerful trees (perhaps 1/5th to 1/3 of them), and most of the trees are set up to shunt out the noise. I suspect that the weights (lagrange multipliers) provide a degree of freedom so that probably reflects the dominance of that particular noise, not unlike an eigenvector/eigenvalue relationship.

Trying to get a clear statement like that out of famous people like Hastie or Tibishirani seems impossible. Maybe because they're afraid they might be wrong.